::: {.callout-tip}
## Here you will learn

The basics of ConScape: importing data, running basic analyses, and exporting resulting maps.
:::

This tutorial is very similar to the notebook in Appendix A from van Moorter et al. (2022). For a broad overview of the ConScape library, please refer to van Moorter et al. (2022). 

In this first notebook we demonstrate the basic workflow as presented in van Moorter et al. (2022) 
to compute the amount of connected habitat and the movement flow in four steps:
1. data import and Grid creation;
2. computation of the GridRSP;
3. computation of the amount of connected habitat;
4. movement flow in two variants (weighted by quality or by proximity).


# Setup the environment

::: {.callout-warning}
This notebook assumes a running installation of Julia. If you didn't already install Julia go [here](/quick_start.qmd).
:::

## Install ConScape

In the first time we use ConsCape, we need to install the library. This step can be ignored in the afterwards,
unless the user wants to reinstall or update the ConScape library to a new version.

Within the Julia environment, installing ConScape is as simple as:


In [ ]:
#| eval: false

# load Pkg library
using Pkg
# install ConScape
Pkg.add("ConScape")

We will also install other libraries that will be used in this notebook, but they are not
necessary to run ConScape.


In [ ]:
#| eval: false

# complementary libraries, to be used in this notebook
Pkg.add("Rasters")

## Load libraries

We continue, and usually would start, by loading the required libraries.


In [ ]:
# load libraries
using Pkg
using ConScape
using Plots

This step is similar to using the `library()` function in R or the `import` command in Python.

When setting up the environment, it is also useful to setup the path to the folders where the input data are located
and where we want to write the results of our analysis. Here we set the `datadir` data folder to the folder where
the internal ConScape example datasets are saved, after the library is installed.


In [ ]:
#| eval: false

# path to files
# Pkg.activate(joinpath(ENV["HOMEPATH"], ".julia", "packages", "ConScape", "spkWs", "data"))

# set folders
datadir = joinpath(ENV["HOMEPATH"], ".julia", "packages", "ConScape", "spkWs", "data")
outdir = joinpath(ENV["TMP"], "figures")
# created the output folder, if it does not exist
if !isdir(outdir)
    mkdir(outdir)
end

# Step 1: Data import and grid creation

## Import data

We start by importing and checking the input data to be used in ConScape.
The first ConScape function is a helper to read maps in ASCII format, the function `readasc()`:


In [ ]:
datadir = "P:/12084000_egenutvikling_bram_van_moorter/Conscape_development/test/data/"
# read habitat quality raster
hab_qual, meta_q = ConScape.readasc(joinpath(datadir, "hab_qual_1000.asc"))
# read movemement probability raster
mov_prob, meta_p = ConScape.readasc(joinpath(datadir, "mov_prob_1000.asc"))

The function reads the map as a matrix and the meta data from the ASCII grid as a dictionary.
ConScape natively reads ASC files, however, Julia allows easy reading of maps in other file
formats through other libraries, for instance:

**This chunk below is not working!!! Missing file!!**
Wouldn't it be much easier to create a function readrast within ConScape?
Or this would increase the dependencies (to Rasters), which is undesiderable?


In [ ]:
#| eval: false
# load Rasters library
using Rasters
# read habitat quality using Rasters()
hab_qual = Raster(joinpath(datadir, "hab_qual_1000_tif.tif"))

Such a raster can easily be converted to a matrix for use in ConScape:


In [ ]:
#| eval: false
# Replace missing values by NaN and transform into a matrix
hab_qual = replace_missing(hab_qual, NaN)
hab_qual = Matrix(transpose(Array(hab_qual)[:,:,1]))

The meta data can be used to verify that the maps are representing the same geographic domain
(i.e. cell size/resolution, number of rows and columns, xy-coordinates of the lower left corner):

**Which of the following options should we use? They were present in different versions of the notebook**


In [ ]:
values(meta_p)
collect(values(meta_p))[1:end .!= 3]
collect(values(meta_p))[1:end .!= 3] == collect(values(meta_q))[1:end .!= 3]

In [ ]:
#| eval: false
delete!(meta_p, "nodata_value")
delete!(meta_q, "nodata_value")
meta_p == meta_q

To ensure that the cells with values match, we conduct the following check and remove non-matching cells:

**Check here**


In [ ]:
#| eval: true
non_matches = findall(xor.(isnan.(mov_prob), isnan.(hab_qual)))
mov_prob[non_matches] .= NaN
hab_qual[non_matches] .= NaN;

## Create a `Grid` object

Define a `ConScape Grid`:


In [ ]:
adjacency_matrix = ConScape.graph_matrix_from_raster(mov_prob)
g = ConScape.Grid(size(mov_prob)..., 
                    affinities = adjacency_matrix,
                    source_qualities = hab_qual,
                    target_qualities = ConScape.sparse(hab_qual),
                    costs = ConScape.mapnz(x -> -log(x), adjacency_matrix))

A `ConScape Grid` describes a graph from a grid of adjacent cells or pixels. It requires four
main inputs: the quality of each pixel both as a source and as a target, the affinity between `i` and
`j` (i.e. probabilities of moving between adjacent pixels `i` and `j`), and the cost of moving between
between `i` and `j`. However, these four inputs can be reduced, for instance, by considering the
quality of a pixel identical as a source and target, or by defining the cost as a function of the
affinities (e.g. a logarithmic relationship). For our illustration, we introduced those two simplifications and only provided two independent data: the quality of a pixel (identical as source
and as target) and the likelihood of moving between adjacent pixels. The likelihood of moving between adjacent pixels `i` and `j` was derived from a ‘permeability map’, which describes
the permeability of a pixel `i` (and is similar to the conductivity in circuit theory). The function
graph matrix from raster computes the values for an `i` − `j` pair from the map either by
the average permeability of `i` and `j` (AverageWeight) or by the permeability of the target pixel
`j` (TargetWeight; the default); the neighbors of a pixel can be defined either as rook (N4)
or as queen (N8; the default).

From the `Grid`, we can plot the qualities of the pixels:


In [ ]:
ConScape.heatmap(g.source_qualities, yflip = true, 
                    title = "Map of habitat uality", 
                    color = cgrad([:white, :green]))
# savefig("figure_grid_outdeg.png")

And the permeability:


In [ ]:
ConScape.plot_outdegrees(g, title = "Map of permeability to movement", color = cgrad(:acton))

In [ ]:
typeof(g.source_qualities)

In [ ]:
typeof(g.affinities)
typeof(g.costmatrix)
typeof(g.costmatrix)

In [ ]:
(g.nrows, g.ncols, g.nrows*g.ncols)

# Step 2: GridRSP creation


In [ ]:
@time h = ConScape.GridRSP(g, θ = 1.0)

Compute derived metrics.
Distance from all pixels to a given target pixel.

Pixel 4300 is our target:


In [ ]:
#| eval: false

tmp = zeros(10179)#5345) #g.nrows * g.ncols
tmp[4300] = 1
ConScape.plot_values(g, tmp, title = "One target pixel t")

Ecological distances from all s to t are:


In [ ]:
dists = ConScape.expected_cost(h)
ConScape.plot_values(g, dists[:,4300], title = "Ecological distances to target pixel t")

Proximity instead of distance


In [ ]:
ConScape.plot_values(g, map!(x -> exp(-x/75), dists[:,4300], dists[:,4300]), 
                        title = "Proximity to target pixel t")

# Computation of habitat functionality

Compute


In [ ]:
func = ConScape.connected_habitat(h, 
                    connectivity_function = ConScape.expected_cost,
                    distance_transformation=x -> exp(-x/75));

# func = ConScape.connected_habitat(h, distance_transformation=x -> exp(-x/75));

Plot


In [ ]:
ConScape.heatmap(Array(func), yflip = true, title = "Map of habitat functionality")

In [ ]:
sum(func)
sum(filter(!isnan, func))

# Movement flow


In [ ]:
ConScape.heatmap(ConScape.betweenness_qweighted(h), yflip = true, title = "g")